# Notebook for blog post
# [A complete guide to preprocess Instagram post data](https://geo.rocks/post/preprocessing-instagram-data)
# Repository: https://github.com/do-me/fast-instagram-scraper
# Contact: dominik@geo.rocks

# Part 0: Concat all csv files in directory with powershell 
## (after mining data with [Fast Instagram Scraper](https://github.com/do-me/fast-instagram-scraper) )
## Automatically taking care of headers and encoding 

```
Get-ChildItem -Filter *.csv | Select-Object -ExpandProperty FullName | Import-Csv  -Encoding UTF8| Export-Csv .\merged\merged.csv -NoTypeInformation -Append -Encoding UTF8
```

# Part 1: Extract Hashtags and clean dataframe
Note: this notebook was tested with a 120 mb csv file containing ~ 180 000 rows with post information. Everything worked flawlessly and fast (some seconds per cell on an i7 machine) 

In [ ]:
# import packages
import geopandas as gpd
import shapely.wkt
import pandas as pd 
from collections.abc import Iterable
from ast import literal_eval
import os 
os.environ['PROJ_LIB'] = 'C:/Users/username/Anaconda3/Library/share/proj' # change this file path if needed

# load merged csv-file in pandas dataframe
df = pd.read_csv("merged.csv",encoding="utf-8")

# define some functions 

# cleaner, removes unnecessary upper nodes
posts_without_text = 0 # just in case you can check later
def str_to_obj(x):
    global posts_without_text
    try:
        return literal_eval(x)[0]["node"]["text"]
    except:
        posts_without_text += 1
        return ""

# simple yet effective list flattener
def flatten(l): #[["k","k"],"k"] -> ['k', 'k', 'k']
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

# some stop characters but could also be entire stopwords such as "and" or "the" - should be done later though
stop_characters = [""," "]

# extracts hashtags from a string (see cell below for example)
def extract_hashtags(x):
    hashtags_space_separated = [i for i in x.split() if i.startswith("#")] # normal hashtags #life #love -> #life, #love
    hashtags_without_space = list(flatten([i.split("#") if i.count("#") > 1 else i for i in hashtags_space_separated])) # without space #life#love -> [#life, #love]
    remove_hashtags = [i.replace("#","") for i in hashtags_without_space] # remove hashtags
    remove_stop_characters = [i for i in remove_hashtags if i not in stop_characters] # remove empty items and space
    return remove_stop_characters

df["edge_media_to_caption.edges"] = df["edge_media_to_caption.edges"].apply(lambda x: str_to_obj(x)) # "denode" text
df["hashtags"] = df["edge_media_to_caption.edges"].apply(lambda x: list(set(extract_hashtags(x)))) # extract unique hashtags by using set
df["location_latlong_str"] = df["location_latlong"] # keep a column copy for later
df # print df

In [ ]:
extract_hashtags("people look at this #amazing #super#fantastic #greenpark https://sample.com/#nohashtag this#hashtagwillbelost")

In [ ]:
posts_without_text/len(df) # in my case 8.5% without text and hashtags

# Part 2: Create and export a simple geodataframe 

In [ ]:
# create geodataframe from pandas dataframe

# prepare some columns
df["location_latlong"] = df["location_latlong"].apply(lambda x: literal_eval(x)) # convert to object
df["lat"] = df["location_latlong"].apply(lambda x: x[0]) # new col lat
df["long"] = df["location_latlong"].apply(lambda x: x[1]) # new col long

# pandas df to geopanas gdf
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(y = df.lat, x = df.long))
gdf.crs = ("EPSG:4326") # set EPSG information...
gdf = gdf.to_crs("EPSG:3857") # ...and reproject to web mercator

# et voilà: a nice, reprojected geodataframe
gdf

In [ ]:
# GeoJSON format cannot store object data types, so all columns must be plain numbers or strings

# drop unnecessary column which cannot be saved due to list type
gdf = gdf.drop(columns=['location_latlong'])

# list to plain string 
gdf["hashtags"] = gdf["hashtags"].apply(', '.join)

# save as GeoJSON but note that you have a GeoJSON with plenty of posts for the same points which might not be what you want
gdf.to_file('all_posts_original.geojson', driver='GeoJSON', encoding="utf-8")

# Part 3: Create geodataframe for unique points only without post information - small size 

In [ ]:
# just unique locations coordinate, slug etc. but no post info
# groupby does the trick here 
uf = df.groupby(['location_latlong_str','location_id','location_name','location_slug'], as_index=False).count()

# rest as above
uf = uf[["location_latlong_str","location_id","location_name","location_slug"]]
uf["location_latlong_str"] = uf["location_latlong_str"].apply(lambda x: literal_eval(x)) # convert to object
uf["lat"] = uf["location_latlong_str"].apply(lambda x: x[0]) 
uf["long"] = uf["location_latlong_str"].apply(lambda x: x[1])    
ugdf = gpd.GeoDataFrame(uf, geometry=gpd.points_from_xy(y = uf.lat, x = uf.long))
ugdf = ugdf.drop(columns=['location_latlong_str'])
ugdf.crs = ("EPSG:4326")
ugdf = ugdf.to_crs("EPSG:3857")
ugdf.to_file('unique_points_no_post_information.geojson', driver='GeoJSON', encoding="utf-8")

# for saving as geopackage
# ugdf=ugdf.loc[ugdf.lat.notnull()] # if needed remove null values
# ugdf["hashtags"] = ugdf["hashtags"].apply(', '.join) # can't save object, must be plain string
# without_nan_ugdf.to_file("unique_points.gpkg", layer='unique_points', driver="GPKG") # throws error on Widnows but works nonetheless

# Part 4: Create geodataframe for unique points only with post information - large size

In [ ]:
# get unique coords and their posts, useful for filtering
un = pd.DataFrame(df.groupby('location_latlong_str'))#.filter(lambda group: len(group) == 1)
un[0] = un[0].apply(lambda x: literal_eval(x)) # convert to object
un["lat"] = un[0].apply(lambda x: x[0]) 
un["long"] = un[0].apply(lambda x: x[1])    

# dataframes in dataframe column 1
un[1] = un[1].apply(lambda x: x.to_json()) # need to transform pandas dataframes to json 

postgdf = gpd.GeoDataFrame(un, geometry=gpd.points_from_xy(y = un.lat, x = un.long))

postgdf = postgdf.drop(columns=[0])
postgdf.crs = ("EPSG:4326")
postgdf = postgdf.to_crs("EPSG:3857")

# rename columns
postgdf.columns = ["posts","lat","long", "geometry"]
postgdf.to_file('unique_points_with_post_information.geojson', driver='GeoJSON', encoding="utf-8")
postgdf
# if AttributeError: 'int' object has no attribute 'encode' 
# column names cant be int must be str!